In [55]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [56]:
pip install sentence_transformers


Note: you may need to restart the kernel to use updated packages.


In [57]:
pip install torch sentencepiece transformers accelerate chromadb tiktoken streamlit fastapi uvicorn python-multipart aiofiles pydantic requests beautifulsoup4 nltk sklearn pandas numpy uvicorn python-multipart aiofiles 

  Using cached fastapi-0.120.4-py3-none-any.whl.metadata (28 kB)
  Using cached beautifulsoup4-4.14.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [58]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [59]:
print("Hello, World!")

Hello, World!


In [60]:
#Load Important Libraries 
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains.summarize import load_summarize_chain
from transformers import T5Tokenizer , T5ForConditionalGeneration
from transformers import pipeline
import torch
import base64 


In [61]:
pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [62]:
pip install hf_xet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Model and Tokenizer
checkpoint = "MBZUAI/LaMini-Flan-T5-248M"
tokenizer = T5Tokenizer.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint, device_map='auto', torch_dtype=torch.float16)

def file_preprocessing(file):
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    text = text_splitter.split_documents(pages)
    final_text = ""
    for doc in text:
        print(doc)
        final_text += doc.page_content + "\n"
    return final_text

#define llm pipeline
def llm_pipeline(filepath):

    pipeline_summarization = pipeline(
        "summarization", 
        model=model, 
        tokenizer=tokenizer, 
        max_length=500,  # Fixed typo
        min_length=50
    ) 
    
    input_text = file_preprocessing(filepath)
    summary = pipeline_summarization(input_text)
    summary = summary[0]['summary_text']  # Fixed: should be summary[0], not input_text[0]
    return summary

# Now you can call the function with an actual filepath
# Example usage:
# result = llm_pipeline("your_file.pdf")
# print(result)

In [67]:
@st.cache_data
def display_pdf(file):
    with open(file, "rb") as f:
        base64_pdf = base64.b64encode(f.read()).decode('utf-8')
    pdf_display = f'<iframe src="data:application/pdf;base64,{base64_pdf}" width="700" height="1000" type="application/pdf"></iframe>'
    st.markdown(pdf_display, unsafe_allow_html=True)

2025-11-01 19:22:58.933 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [ ]:
#Streamlit App Code
st.set_page_config(page_title="Document Summarizer")
def main():
    st.title("Document Summarizer using LLM")
    st.write("Upload a PDF document to generate its summary.")
    uploaded_file = st.file_uploader("Choose a PDF file", type=["pdf"])
    if uploaded_file is not None:
        if st.button("Generate Summary"):
            col1, col2 = st.columns(2)
            with col1:
                st.header("Uploaded Document")
                display_pdf(uploaded_file)
            with col2:
                st.header("Document Summary")
                summary = llm_pipeline(uploaded_file)
                st.write(summary)
    
if __name__ == "__main__":  
    main()
    
            
            



2025-11-01 19:34:44.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
